In [45]:
from enum import Enum

In [46]:
import numpy as np

In [47]:
def _process_raw_tile(raw_tile):
    tile_id = int(raw_tile.split("\n")[0].replace("Tile ", "").replace(":", ""))
    tile_data = np.array(tuple(tuple(1 if val == "#" else 0 for val in row) for row in raw_tile.split("\n")[1:] if row))
    return tile_id, tile_data

with open("../input.txt", mode="r") as file_pointer:
    tiles = dict(_process_raw_tile(raw_tile) for raw_tile in file_pointer.read().split("\n\n") if raw_tile)

In [48]:
class Directions(Enum):
    up = (-1, 0)
    down = (1, 0)
    right = (0, 1)
    left = (0, -1)

In [49]:
def tile_border(tile, direction):
    if direction in (Directions.up, Directions.up.value):
        border = (0, slice(None))
    elif direction in (Directions.down, Directions.down.value):
        border = (-1, slice(None))
    elif direction in (Directions.right, Directions.right.value):
        border = (slice(None), -1)
    elif direction in (Directions.left, Directions.left.value):
        border = (slice(None), 0)
    else:
        raise KeyError(direction)
    return tile[border]


def gen_borders(tile):
    for direction in Directions:
        yield tile_border(tile, direction)
        
def gen_all_borders(tile):
    """
    Yields all borders, including the flipped ones.
    """
    for border in gen_borders(tile):
        yield border
        yield border[::-1]

In [50]:
tiles_borders = {
    key: tuple(gen_all_borders(tile)) for key, tile in tiles.items()
}

In [51]:
def compatible_borders(borders_1, borders_2):
    return any((border_1 == border_2).all() for border_2 in borders_2 for border_1 in borders_1)

In [52]:
compatible_tiles = {
    tile_id_1: set(tile_id_2 for tile_id_2, borders_2 in tiles_borders.items() if tile_id_1 != tile_id_2 and compatible_borders(borders_1, borders_2))
    for tile_id_1, borders_1 in tiles_borders.items()
}

In [53]:
def gen_corner_tile_ids(compatible_tiles):
    for tile_id, compatible_tile_ids in compatible_tiles.items():
        if len(compatible_tile_ids) == 2:
            yield tile_id

In [54]:
tuple(gen_corner_tile_ids(compatible_tiles))

(1117, 1543, 1213, 1291)

In [55]:
import math

In [56]:
def create_empty_image(n_tiles):
    dim = int(math.sqrt(n_tiles))
    return {(x, y): None for y in range(dim) for x in range(dim)}

In [57]:
image = create_empty_image(len(tiles))

In [58]:
def gen_neighbor_positions(position):
    assert len(position) == 2
    for direction in Directions:
        direction = direction.value
        yield direction, (position[0] + direction[0], position[1] + direction[1])
    
def gen_neighbors(position, image):
    for direction, neighbor_position in gen_neighbor_positions(position):
        if neighbor_position in image:
            yield direction, neighbor_position, image[neighbor_position]

In [59]:
availabe_corner_tile_ids = set(gen_corner_tile_ids(compatible_tiles))
availabe_non_corner_tile_ids = set(tiles.keys())
availabe_non_corner_tile_ids -= availabe_corner_tile_ids

In [60]:
first_corner_tile = availabe_corner_tile_ids.pop()

In [61]:
image[(0, 0)] = first_corner_tile

In [62]:
image

{(0, 0): 1213,
 (1, 0): None,
 (2, 0): None,
 (3, 0): None,
 (4, 0): None,
 (5, 0): None,
 (6, 0): None,
 (7, 0): None,
 (8, 0): None,
 (9, 0): None,
 (10, 0): None,
 (11, 0): None,
 (0, 1): None,
 (1, 1): None,
 (2, 1): None,
 (3, 1): None,
 (4, 1): None,
 (5, 1): None,
 (6, 1): None,
 (7, 1): None,
 (8, 1): None,
 (9, 1): None,
 (10, 1): None,
 (11, 1): None,
 (0, 2): None,
 (1, 2): None,
 (2, 2): None,
 (3, 2): None,
 (4, 2): None,
 (5, 2): None,
 (6, 2): None,
 (7, 2): None,
 (8, 2): None,
 (9, 2): None,
 (10, 2): None,
 (11, 2): None,
 (0, 3): None,
 (1, 3): None,
 (2, 3): None,
 (3, 3): None,
 (4, 3): None,
 (5, 3): None,
 (6, 3): None,
 (7, 3): None,
 (8, 3): None,
 (9, 3): None,
 (10, 3): None,
 (11, 3): None,
 (0, 4): None,
 (1, 4): None,
 (2, 4): None,
 (3, 4): None,
 (4, 4): None,
 (5, 4): None,
 (6, 4): None,
 (7, 4): None,
 (8, 4): None,
 (9, 4): None,
 (10, 4): None,
 (11, 4): None,
 (0, 5): None,
 (1, 5): None,
 (2, 5): None,
 (3, 5): None,
 (4, 5): None,
 (5, 5): None,


In [63]:
def is_corner(position, image):
    dim = int(math.sqrt(len(image)))
    return position[0] in (0, dim - 1) and position[1] in (0, dim - 1)

In [64]:
def calc_possible_tiles(position, image, available_corner_tiles, available_non_corner_tiles):
    if is_corner(position, image):
        possible_tiles = available_corner_tiles
    else:
        possible_tiles = available_non_corner_tiles
    for _, _, neighbor_tile_id in gen_neighbors(position, image):
        if neighbor_tile_id is not None:
            possible_tiles = possible_tiles.intersection(compatible_tiles[neighbor_tile_id])
    return position, possible_tiles


def gen_possible_tiles(image, available_corner_tiles, available_non_corner_tiles):
    for position, tile in image.items():
        if tile is not None:
            continue
        yield calc_possible_tiles(position, image, available_corner_tiles, available_non_corner_tiles)
        
        
def is_solvable(possible_tiles):
    return all(len(val) > 0 for val in possible_tiles.values())

In [65]:
possible_tiles = dict(gen_possible_tiles(image, availabe_corner_tile_ids, availabe_non_corner_tile_ids))

In [66]:
availabe_corner_tile_ids

{1117, 1291, 1543}

In [67]:
is_solvable(possible_tiles)

True

In [68]:
def find_most_constrained_position(possible_tiles):
    return min(possible_tiles, key=lambda key: len(possible_tiles[key]))

In [69]:
find_most_constrained_position(possible_tiles)

(1, 0)

In [70]:
def is_complete(image):
    return not any(tile is None for tile in image.values())

In [71]:
def search(image, available_corner_tiles, available_non_corner_tiles):
    if is_complete(image):
        yield image
        return
    possible_tiles = dict(gen_possible_tiles(image, available_corner_tiles, available_non_corner_tiles))
    if not is_solvable(possible_tiles):
        return
    position = find_most_constrained_position(possible_tiles)
    available_tiles = possible_tiles[position]
    for tile in available_tiles:
        new_image = {**image, position: tile}
        new_available_corner_tiles = available_corner_tiles - set([tile])
        new_available_non_corner_tiles = available_non_corner_tiles - set([tile])
        for solution in search(new_image, new_available_corner_tiles, new_available_non_corner_tiles):
            yield solution

In [72]:
solution = tuple(search(image, availabe_corner_tile_ids, availabe_non_corner_tile_ids))[0]

In [73]:
answer = math.prod(tile_id for position, tile_id in solution.items() if is_corner(position, solution))

In [74]:
answer

2699020245973

In [75]:
assert answer == 2699020245973

In [76]:
from itertools import product

In [77]:
tuple(product(range(4), (False, True), (False, True)))

((0, False, False),
 (0, False, True),
 (0, True, False),
 (0, True, True),
 (1, False, False),
 (1, False, True),
 (1, True, False),
 (1, True, True),
 (2, False, False),
 (2, False, True),
 (2, True, False),
 (2, True, True),
 (3, False, False),
 (3, False, True),
 (3, True, False),
 (3, True, True))

In [78]:
def calc_variation(tile, flip_0, flip_1, rotation):
    if flip_0:
        tile = np.flip(tile, 0)
    if flip_1:
        tile = np.flip(tile, 1)
    return np.rot90(tile, rotation)

def gen_variations(tile):
    for config in tuple(product(range(4), (False, True), (False, True))):
        yield calc_variation(tile, *config)
        
INVERSE_DIRECTIONS = {
    Directions.up: Directions.down,
    Directions.down: Directions.up,
    Directions.left: Directions.right,
    Directions.right: Directions.left,
    Directions.up.value: Directions.down,
    Directions.down.value: Directions.up,
    Directions.left.value: Directions.right,
    Directions.right.value: Directions.left,
}
        
def match_border(tile_1, tile_2, direction):
    return (tile_border(tile_1, direction) == tile_border(tile_2, INVERSE_DIRECTIONS[direction])).all()


def gen_matching_variations(tile_1, tile_2, direction):
    for var_2 in gen_variations(tile_2):
        if match_border(tile_1, var_2, direction):
            yield var_2

In [79]:
new_image = {key: None for key in solution}

In [80]:
tuple(gen_matching_variations(tiles[1213], tiles[3373], Directions.right))

()

In [81]:
def calc_constrains(image):
    return {
        position: len(tuple(neighbor for _, _, neighbor in gen_neighbors(position, image) if neighbor is not None))
        for position, tile in image.items()
        if tile is None
    }

def find_most_constrained_position(constrains):
    return max(constrains, key=constrains.get)

In [82]:
def search_2(image, old_image):
    """
    Search, but now for the variation of each tile.
    """
    if is_complete(image):
        yield image
        return
    constrains = calc_constrains(image)
    position = find_most_constrained_position(constrains)
    tile = tiles[old_image[position]]
    possible_variations = tuple(gen_variations(tile))
    for direction, neighbor_position, neighbor_tile in gen_neighbors(position, image):
        if neighbor_tile is None:
            continue
        possible_variations = tuple(variation for variation in gen_matching_variations(neighbor_tile, tile, INVERSE_DIRECTIONS[direction]) if any((variation == other).all() for other in possible_variations))
    for variation in possible_variations:
        new_image = {**image, position: variation}
        for solution in search_2(new_image, old_image):
            yield solution

In [83]:
start_position = (0, 0)
new_image = {key: None for key in solution}
new_image[start_position] = tiles[solution[start_position]]
tuple(search_2(new_image, solution))

()

In [84]:
from itertools import islice

In [85]:
start_position = (0, 0)
for i, variation in enumerate(gen_variations(tiles[solution[start_position]])):
    print(i)
    new_image = {key: None for key in solution}
    new_image[start_position] = variation
    solutions = tuple(islice(search_2(new_image, solution), 1))
    if solutions:
        solution = solutions[0]
        break

0
1


In [86]:
solution[(0, 0)]

array([[1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0, 1, 1, 1]])

In [87]:
solution[(0, 1)]

array([[0, 1, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

In [88]:
solution[(1, 1)]

array([[1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 1, 1, 1, 0, 0, 0]])

In [89]:
def remove_border(tile):
    return tile[1:-1, 1:-1]

In [90]:
solution = {position: remove_border(tile) for position, tile in solution.items()}

In [93]:
solution[(0, 0)]

array([[0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 1]])

In [ ]:
sol = tuple(tuple(solution[()]))

In [ ]:
np.stack()

In [97]:
dim = int(math.sqrt(len(solution)))

In [99]:
tuple(np.hstack(solution[(x, y)] for y in range(dim)) for x in range(dim))

<ipython-input-99-ed994ed5e1dc>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  tuple(np.hstack(solution[(x, y)] for y in range(dim)) for x in range(dim))


(array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
         1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
         0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
         1, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 1, 0, 1, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
         0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0]

In [ ]:
tuple(diagonal_walk(new_image))

In [ ]:
def search_2(new_image, old_image):
    if is_complete(new_image):
        yield new_image
        return
    

In [ ]:
for var_1 in gen_variations(tiles[1213]):
    for var_2 in gen_variations(tiles[3373]):
        if match_border(var_1, var_2, Directions.right):
            print("yes!")

In [ ]:
for var_1 in gen_variations(tiles[1213]):
    for var_2 in gen_variations(tiles[1297]):
        if match_border(var_1, var_2, Directions.down):
            print("yes!")

In [ ]:
print(tiles[1213])

In [ ]:
print(tiles[3373])

In [ ]:
print(np.rot90(tiles[1213], 1))

In [ ]:
print(np.rot90(tiles[3373], 1))

In [ ]:
solution[(0, 2)]

In [ ]:
print(np.flip(np.rot90(tiles[1039]), -1), 0)